### Set up the right directory for importing modules

In [ ]:
import matplotlib.pyplot as plt #to remove later 
import sys
from pathlib import Path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
import src.CA_model as CA
import src.analysis as an
import src.utils as ut
import numpy as np
import matplotlib.pyplot as plt
import time
from importlib import reload


In [ ]:
import src.CA_model as CA
import src.analysis as an
import time
import matplotlib.pyplot as plt
import numpy as np
import importlib

In [ ]:
# run to reload CA_model.py and analysis.py and utils.py for updated code
reload(CA)
reload(an)
reload(ut)

#### Parameter settings

In [ ]:
size = 100                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_frac=0.2                       # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=10                                # radius of neighbourhood
N_steps=200                         # number of iterations
skip=0                              # iterations to skip (equilibration period)
seed=1000

#### Generate grids for analysis

In [ ]:
grids = CA.evolve_CA(
    size=size,
    p=p,
    update_rule=update_rule,
    true_frac=true_frac,
    k=k,
    M=M,
    N_steps=N_steps,
    skip=skip,
    seed=seed,
)

#### Plot starting and final grid

In [ ]:
an.show_grids([grids[0], grids[-1]], [skip, N_steps])

#### Create GIF Animation

In [ ]:
an.animate_grids(grids, dpi=100)

<<<<<<< local
#### Save data
=======
an.plot_cluster_size_distr(size_list=size_list, fit=fit) #Cumulative distirbution
>>>>>>> remote

In [ ]:
# if the equilibration period was not skipped yet, correct for that before saving
if skip == 0:
    skip = 100
    grids = grids[skip:]

ut.save_data(grids, size, update_rule, true_frac, k, M, N_steps, skip, seed=seed)

#### Load data

In [ ]:
loaded_grids = ut.load_data(size, update_rule, true_frac, k, M, N_steps, skip, seed=seed)

# check that the saved data is correctly recovered from the original data
assert np.all(loaded_grids == grids)

#### Plot cluster size distribution

In [ ]:
size_list, fit = an.cluster_sizes(loaded_grids)
alpha = fit.truncated_power_law.alpha
s_char = 1 / (fit.truncated_power_law.Lambda)
R, p = fit.distribution_compare("truncated_power_law", "exponential", normalized_ratio=True)

print("Scaling exponent: ", alpha)
print("Characteristic length: ", s_char)
print("Loglikelihood ratio (if positive, (truncated) power law more likely than exponential): ", R)
print("Significance value: ", p)

In [ ]:
fig = an.plot_cluster_size_distr(size_lists=[size_list], fits=[fit])

## Averaging over several datasets

In [ ]:
size = 200                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_frac=0.2                       # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=10                                # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)

N_evolutions = 5                    # number of full evolutions to generate for this set of parameters
all_grids = []

for seed in range(N_evolutions):
    start = time.time()
    grids = CA.evolve_CA(
        size=size,
        p=p,
        update_rule=update_rule,
        true_frac=true_frac,
        k=k,
        M=M,
        N_steps=N_steps,
        skip=skip,
        seed=seed,
    )
    all_grids.append(grids)
    end = time.time()
    print(f"Grid evolution {seed+1} out of {N_evolutions} completed in {end-start} seconds.")

In [ ]:
# flatten the list of lists into a 1D list of grids
combined_grids = [grid for grid_list in all_grids for grid in grid_list]

In [ ]:
size_list, fit = an.cluster_sizes(combined_grids)
alpha = fit.truncated_power_law.alpha
s_char = 1 / (fit.truncated_power_law.Lambda)
R, p = fit.distribution_compare("truncated_power_law", "exponential", normalized_ratio=True)

print("Scaling exponent: ", alpha)
print("Characteristic length: ", s_char)
print("Loglikelihood ratio (if positive, (truncated) power law more likely than exponential): ", R)
print("Significance value: ", p)

In [ ]:
an.plot_cluster_size_distr(size_list=[size_list], fit=[fit])

## Comparing different sets of parameters

#### Generating data

In [ ]:
size = 500                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_fracs=np.arange(0.05,0.7,0.05) # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=5                                 # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)

# all_grids = []

for i in range(len(true_fracs)):
    start = time.time()
    grids = CA.evolve_CA(
        size=size,
        p=p,
        update_rule=update_rule,
        true_frac=true_fracs[i],
        k=k,
        M=M,
        N_steps=N_steps,
        skip=skip,
        seed=i,
    )
    # all_grids.append(grids)
    end = time.time()
    print(f"Grid evolution {i+1} out of {len(true_fracs)} completed in {end-start} seconds.")

    ut.save_data(grids, size, update_rule, true_frac, k, M, N_steps, skip, seed)


#### Analysis

In [ ]:
# parameters for which data should be analysed
size = 500                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_fracs=np.arange(0.05,0.7,0.05) # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=5                                 # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)

size_lists = []
fits = []

for i in range(len(true_fracs)):
    loaded_grids = ut.load_data(size, update_rule, np.round(true_fracs[i],2), k, M, N_steps, skip, seed)
    # retrieve the cumulative cluster size distribution
    size_list, fit = an.cluster_sizes(loaded_grids)
    size_lists.append(size_list)
    fits.append(fit)

In [ ]:
an.plot_cluster_size_distr(size_lists=size_lists, fits=fits)

In [ ]:
# parameters for which data should be analysed
size = 500                          # width and height of the grid
p = 0.5                             # starting fraction of vegetation
update_rule = CA.update_Scanlon2007 # function containing update rule
true_fracs=np.arange(0.05,0.7,0.05) # 'natural' (equilibrium) fraction of vegetation
k=3                                 # strength of local interactions
M=10                                 # radius of neighbourhood
N_steps=200                         # number of iterations
skip=100                            # iterations to skip (equilibration period)
seed=0

size_lists = []
fits = []

for i in range(len(true_fracs)):
    loaded_grids = ut.load_data(size, update_rule, np.round(true_fracs[i],2), k, M, N_steps, skip, seed)
    # retrieve the cumulative cluster size distribution
    size_list, fit = an.cluster_sizes(loaded_grids)
    size_lists.append(size_list)
    fits.append(fit)

In [ ]:
an.plot_cluster_size_distr(size_lists=size_lists, fits=fits)

## Scalon rule with different weight on the local and global rule 

In [ ]:

"""
Difference rules weight """

#simulation is really slow 

phi_values = [0.0, 0.25, 0.5, 0.75, 1] #the different weight of the local rule 
N_evolutions = 5
results = {}


for phi in phi_values: 
    print(f"\nRunning Scalon_exp with phi = {phi}")
    all_grids = []

    for seed in range(N_evolutions): #we run 5 independent CA evolutions for each phi
        start = time.time() #function that returns the time until another point, measure performance 
        grids = CA.evolve_CA(
            size = size, #change to 200 or to paralize thing 
            p = p, 
            update_rule = CA.update_Scanlon_exp, 
            true_frac=true_frac, 
            k=k,
            M = 10, 
            N_steps = N_steps, #the only parameter that changes 
            skip = skip,
            seed = seed , 
            phi = phi
        )
        all_grids.append(grids)
        end = time.time()
        print(f"Evolution {seed+1}/{N_evolutions} completed in {end-start: .2f} s'")

    #Flatten all grids
    combined_grids = [g for grid_list in all_grids for g in grid_list] #this is only used to analayse the different grids but putting them next to each other? 

    #Compute cluster sizes and fit 
    size_list, fit = an.cluster_sizes(combined_grids) #size_list gives a list of all cluster sizes, and fit models the distribution 
    results[phi] = (size_list, fit)

    #Print parameters
    beta = fit.truncated_power_law.alpha 
    s_char = 1 /fit.truncated_power_law.Lambda
    R, p_val = fit.distribution_compare("truncated_power_law", "exponential") # these are just statistical test to compare the 2 model, if R>0 -> the truncated power law fits better, if p<0.05 then the difference between the fit of the 2 models is statistically significant 

    print(f" beta = {beta:.3f}") 
    print(f" s_char = {s_char:.3f}") 
    print(f" R = {R:.3f}, p = {p_val:.3f}") 
    
    # Plot 
    an.plot_cluster_size_distr([size_list], [fit])



In [ ]:

"""
Difference rules weight- optimized for faster simulations
Only keeps the last grid instead of storing all the grids for statistics """


phi_values = [0.0, 0.25, 0.5, 0.75, 1] #the different weight of the local rule 
N_evolutions = 4
results = {}
final_grids_by_phi = {} #for visualition

for phi in phi_values: 
    print(f"\nRunning Scalon_exp with phi = {phi}")
    final_grids = []

    for seed in range(N_evolutions): #we run 5 independent CA evolutions for each phi
        start = time.time() #function that returns the time until another point, measure performance 
        grids = CA.evolve_CA(
            size = 90, 
            p = p, 
            update_rule = CA.update_Scanlon_exp, 
            true_frac=true_frac, 
            k=k,
            M = 10, 
            N_steps = 80, 
            skip = 60,
            seed = seed , 
            phi = phi
        )
        final_grids.append(grids[-1])
        end = time.time()
        print(f"Evolution {seed+1}/{N_evolutions} completed in {end-start: .2f} s'")

    final_grids_by_phi[phi] = final_grids 

    #Compute cluster sizes and fit 
    size_list, fit = an.cluster_sizes_safe(final_grids) #size_list gives a list of all cluster sizes, and fit models the distribution 
    
    if len(size_list) ==0: 
        print("No clusters large enough for fitting")
        results[phi]= (size_list,None)
        continue
    
    results[phi] = (size_list, fit)

    #Print parameters
    beta = fit.truncated_power_law.alpha 
    s_char = 1 /fit.truncated_power_law.Lambda
    R, p_val = fit.distribution_compare("truncated_power_law", "exponential") # these are just statistical test to compare the 2 model, if R>0 -> the truncated power law fits better, if p<0.05 then the difference between the fit of the 2 models is statistically significant 

    print(f" beta = {beta:.3f}") 
    print(f" s_char = {s_char:.3f}") 
    print(f" R = {R:.3f}, p = {p_val:.3f}") 
    
    # Plot cluster size distribution for each phi 
    plt.figure(figsize=(6,4))
    an.plot_cluster_size_distr([size_list], [fit])
    plt.title(f"Cluster size distribution (phi = {phi})")
    plt.show()


#Visualizing the final grids
plt.figure(figsize=(12, 8))

for i, phi in enumerate(phi_values):
    grid = final_grids_by_phi[phi][0]  # first seed's final grid

    plt.subplot(2, 3, i+1)
    plt.imshow(grid, cmap="Greens")
    plt.title(f"Final grid (phi = {phi})")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
"""
P value as a function of phi
We want to see when does the p value becomes statistically significant (0.05) to say that it's a power law, depending on phi 

"""



## Percolation

In [ ]:
"""
Percolation vs rainfall
We calculate the probablity of percolation for different 
rainfall percentage (true_frac)
"""
rain_values = [0.0, 0.2, 0.4, 0.6, 0.8]
N_evolutions = 20 #20simulations with different seeds 
percs= []

for r in rain_values: 
    count = 0
    for seed in range(N_evolutions): 
        grids = CA.evolve_CA(
            size = 100, 
            p = 0.5, 
            update_rule=CA.update_Scanlon_exp, 
            true_frac=r, 
            k=3, 
            M=10, 
            N_steps=200, 
            skip=100, 
            seed=seed, 
            phi=0.5 ) 
        last_grids = grids[:, 100] #taking more grids final 100 into consideration, and take 
        
        if an.has_vertical_percolation(last_grids):
            if an.has_horizontal_percolation():
                count +=1
    percs.append((count/100) / N_evolutions) #divide by the number of grids we take into consideration, and the number of simulation 

plt.plot(rain_values, percs, marker = 'o')
plt.xlabel("Rainfall (tru_frac)")  
plt.ylabel("Percolation probability") 
plt.title("Percolation threshold") 
plt.show()


In [ ]:
"""
Percolation vs phi (local/global rules) see below percolation trehsold the local rule shows percolation
"""

phi_values = [0.0, 0.25, 0.5, 0.75, 1]
rain_values = [0.3, 0.5, 0.7] #only around the percolation treshold 
N_evolutions = 5 
percs = []

for rain in rain_values: 
    percs = []
    for phi in phi_values: 
        count = 0
        for seed in range(N_evolutions): 
            grids = CA.evolve_CA(
                size = 100, 
                p = p, 
                update_rule=CA.update_Scanlon_exp, 
                true_frac=rain, 
                k=k, 
                M=10, 
                N_steps=100, 
                skip=50, 
                seed=seed, 
                phi=phi) 
            final_grid = grids[-1] 
                            
            if an.has_vertical_percolation(final_grid): 
                count +=1
        percs.append(count / N_evolutions)
    plt.plot(phi_values, percs, marker='o', label=f"Rain = {rain}" )

plt.xlabel("Phi (weight of local rule)")  
plt.ylabel("Percolation probability") 
plt.title(f"Percolation vs Phi (rain fraction = {true_frac})") 
plt.legend()
plt.show()
#percolation_probability = sum(percs)/N_evolutions
#print("Percolation has a probability of ", percolation_probability)

In [ ]:
"""
Cluster size near the percolation treshold"""

rain_values = [0.4, 0.5, 0.6]  # around treshold region
phi = 0.5
N_evolutions = 5

for rain in rain_values:
    final_grids = []

    for seed in range(N_evolutions):
        grids = CA.evolve_CA(
            size=100,
            p=p,
            update_rule=CA.update_Scanlon_exp,
            true_frac=rain,
            k=k,
            M=10,
            N_steps=100,
            skip=50,
            seed=seed,
            phi=phi
        )
        final_grids.append(grids[-1])

    size_list, fit = an.cluster_sizes(final_grids)
    size_list = [s for s in size_list if s >=5] #only keeps the clusiter sizes >= 5 to fit power law better

    print(f"\nRainfall = {rain}")
    if fit is not None and len(size_list)>0:
        beta = fit.truncated_power_law.alpha
        print(f" alpha = {beta:.3f}")
        an.plot_cluster_size_distr([size_list], [fit])
    else:
        print("No large clusters.")
